Train MLP for all fingers!\
Using hyper param optimalization
Data source: \
sliding windowed powers for mu and beta band\
Subject 1

In [1]:
import os
import numpy as np
import h5py
from scipy import stats
import scipy.io
import mne

mne.set_log_level('error')

from random import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold


import torch
import torch.nn as nn
import torch.optim as optim

import optuna


from utils.load import Load
from config.default import cfg

%load_ext autoreload
%autoreload 2


In [21]:
subject_data = {}
# Load the data  from the HDF5 file
target_dir = 'features'
tag = 'reproduced_with_bad'

for subject in cfg['subjects']:
    file_path = os.path.join(target_dir, tag+'_'+subject + '.h5')

    data = {}
    with h5py.File(file_path, 'r') as h5file:
        for key in h5file.keys():
            data[key] = np.array(h5file[key])

    subject_data[subject] = data


for subject_id in subject_data:
    print(subject_id)
    print(subject_data[subject_id].keys())

S1
dict_keys(['index', 'little', 'middle', 'ring', 'thumb'])
S2
dict_keys(['index', 'little', 'middle', 'ring', 'thumb'])
S3
dict_keys(['index', 'little', 'middle', 'ring', 'thumb'])
S4
dict_keys(['index', 'little', 'middle', 'ring', 'thumb'])
S5
dict_keys(['index', 'little', 'middle', 'ring', 'thumb'])


In [23]:
X = []
y = []
# Loop through each subject in the dataset
for subject_id, (subject, data) in enumerate(subject_data.items()):

    features = np.concatenate(list(data.values()), axis=0)
    features = features.reshape(features.shape[0], -1)

    features = np.insert(features, 0, subject_id, axis=1) # add subject id as first column
    labels = np.concatenate([np.ones(data[finger].shape[0]) * i for i, finger in enumerate(data)], axis=0)
   
    X += [features]
    y += [labels]

X = np.concatenate(X, axis=0)
y = np.concatenate(y, axis=0)

print(X.shape)
print(y.shape)

(1250, 8217)
(1250,)


In [3]:
class SingleLayerMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, activation):
        super(SingleLayerMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.activation = activation
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = nn.Softmax(dim=1)(x)
        return x

In [26]:
def train(X_train, y_train, X_test, y_test, model, criterion, optimizer, num_epochs=100):
    X_train = torch.tensor(X_train, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.long)
    y_test = torch.tensor(y_test, dtype=torch.long)


    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        # # Shuffle X and y together
        p = np.random.permutation(len(X_train))
        X_train, y_train = X_train[p], y_train[p]

    with torch.no_grad():
        y_pred = model(X_test)
        y_pred = torch.argmax(y_pred, dim=1)

    acc = accuracy_score(y_test, y_pred)
    return acc

def objective(trial, X, y):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    num_epochs = trial.suggest_int("num_epochs", 100, 2000)
    hidden_size = trial.suggest_int("hidden_size", 16, 128)
    activation_name = trial.suggest_categorical("activation", ["relu", "elu", "leaky_relu"])
    optimizer = trial.suggest_categorical("optimizer", ["SGD", "Adam"])

    if activation_name == "relu":
        activation = nn.ReLU()
    elif activation_name == "elu":
        activation = nn.ELU()
    elif activation_name == "leaky_relu":
        activation = nn.LeakyReLU()

    if optimizer == "SGD":
        optimizer = optim.SGD
    elif optimizer == "Adam":
        optimizer = optim.Adam

    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)
    model = SingleLayerMLP(train_X.shape[1], hidden_size, 5, activation)
    criterion = nn.CrossEntropyLoss()
    optimizer = optimizer(model.parameters(), lr=learning_rate)
    return train(train_X, train_y, test_X, test_y, model, criterion, optimizer, num_epochs=num_epochs)


def train_MLP(X, y):


    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective(trial, X, y), n_trials=10)

    best_trial = study.best_trial

    print(f'Best trial params: {best_trial.params}')
    print(f'Best trial accuracy: {best_trial.value * 100:.2f}%')



In [ ]:
# connect data

In [27]:
train_MLP(X, y)

[I 2023-04-15 16:39:56,675] A new study created in memory with name: no-name-a3257f6f-2c8c-4825-9737-7b5514450793
[I 2023-04-15 16:40:25,675] Trial 0 finished with value: 0.204 and parameters: {'learning_rate': 0.005472281652350529, 'num_epochs': 1250, 'hidden_size': 17, 'activation': 'elu', 'optimizer': 'Adam'}. Best is trial 0 with value: 0.204.
[I 2023-04-15 16:40:52,512] Trial 1 finished with value: 0.248 and parameters: {'learning_rate': 1.6221729607802193e-05, 'num_epochs': 546, 'hidden_size': 112, 'activation': 'elu', 'optimizer': 'SGD'}. Best is trial 1 with value: 0.248.
[I 2023-04-15 16:41:27,953] Trial 2 finished with value: 0.212 and parameters: {'learning_rate': 0.00011273378060661761, 'num_epochs': 1011, 'hidden_size': 79, 'activation': 'leaky_relu', 'optimizer': 'Adam'}. Best is trial 1 with value: 0.248.
[I 2023-04-15 16:41:48,057] Trial 3 finished with value: 0.18 and parameters: {'learning_rate': 5.059915630458249e-05, 'num_epochs': 627, 'hidden_size': 107, 'activatio

Best trial params: {'learning_rate': 1.6221729607802193e-05, 'num_epochs': 546, 'hidden_size': 112, 'activation': 'elu', 'optimizer': 'SGD'}
Best trial accuracy: 24.80%
